# Module 2 - Model Tracking Using MLflow
In this section we discuss how to use [MLflow](https://mlflow.org/docs/latest/index.html) to track model experiments, save experiment and model information / artifacts to the mlflow database, learn about the model registry and choose models for staging and production,

```{admonition} Different MLflow Components
:class: tip
1. Model Tracking - For logging parameters, code versions, metrics etc.
2. MLflow models - A packagng format for easy deployment on systems such as docker, Spark, Databricks etc.
3. Model registry - A centralised model store that helps with the governance of models
4. Projects - Standard format for packaging reusable data science code 
5. Recipes -  Predefined template for building models
```

## Model Tracking: 
MLflow can log pretty much anything you need or think would be important to know about the model build. The different objects that can be tracked are broken down into 4 different distinct areas:

| Name              | Used for                                                          |
| :---------------- | :----------------------------------------------------------------:|
| Parameters        |  Contstant values such as configuration params, model params etc  |
| Metrics           |  Values updated during the run e.g. RMSE, Gini, etc.              |
| Artifacts         |  Files produced by the run (Model weights, pipeline etc)          |
| Additional        |  Items like creators names, tags etc                              |

## That's Great But Where is All The Information Stored?
MLflow runs can be recorded to
1. Local files,
2. To a SQLAlchemy-compatible database, or
3. Remotely to a tracking server. 

By default, the MLflow Python API logs runs locally to files in an mlruns directory wherever you ran your program. You can then run mlflow ui to see the logged runs.
For us, because we are running it locally with a sqlite database, artifacts are stored in the `mlruns directory` and entities are stored in the `mlruns.db`

__To log runs remotely, set the MLFLOW_TRACKING_URI environment variable to a tracking server’s URI or call mlflow.set_tracking_uri().__

```{admonition} Experiments Not Showing Up In The UI
:class: warning
When running MLflow you might notice that the experiments do not show up in the UI even though they are present in the `mlruns` directory. Your issue is probably that you ran the command `mlflow ui` outside of the directory that contains the mlruns folder.

to fix this, simply navigate to the directory that contains the `mlruns` folder and rerun the ui command. Any tests that you have run and that have not been picked up will show up then!
```

In [1]:
import sys

sys.path.append("/home/ubuntu/sh-mlops-zoomcamp/mlops_jupyter_book")
from utils.utils import ROOT_DIR, render_itable, init_jb_table_style, _import_data
from itables import init_notebook_mode
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_diabetes
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import mlflow
from mlflow.models.signature import infer_signature

init_jb_table_style()
init_notebook_mode(all_interactive=True, connected=True)

In [2]:
# Setting the tracking uri database to use and also setting the experiment name
# If the experiment exists then we log in to that experiment. If not, it creates a new experiment
%env MLFLOW_TRACKING_URI  "sqlite:///mlflow.db"
mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("jb-experiment")

env: MLFLOW_TRACKING_URI="sqlite:///mlflow.db"


<Experiment: artifact_location='/home/ubuntu/sh-mlops-zoomcamp/mlruns/1', creation_time=1685052038922, experiment_id='1', last_update_time=1685052038922, lifecycle_stage='active', name='jb-experiment', tags={}>

## Autologging

MLflow has a fantastic auto-logging function that logs everything about the training and testing parameters in a single step. The `autolog` function.

For sklearn (Probably what we all we be using most of the time) it logs the following information:
1. Training score obtained by estimator.score
2. Parameters obtained by estimator.get_params
3. Class name
4. Fitted Estimator


In [3]:
mlflow.autolog()
with mlflow.start_run(run_name="autolog") as run:
    # Load the diabetes dataset.
    db = load_diabetes()
    X_train, X_test, y_train, y_test = train_test_split(db.data, db.target)

    # Create and train models.
    rf = RandomForestRegressor(n_estimators=100, max_depth=6, max_features=3)
    rf.fit(X_train, y_train)

    # Use the model to make predictions on the test dataset.
    predictions = rf.predict(X_test)

    signature = infer_signature(X_test, predictions)
    mlflow.sklearn.log_model(rf, "model", signature=signature)

    print("Run ID: {}".format(run.info.run_id))

2023/06/05 23:33:49 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.
2023/06/05 23:33:52 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/ubuntu/anaconda3/lib/python3.10/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils."


Run ID: d52e57b67cfb43e48b4c3b30fc336dca


In [4]:
mlflow.autolog(disable=True)

with mlflow.start_run(run_name="no_autolog") as run:
    # Load the diabetes dataset.
    db = load_diabetes()
    X_train, X_test, y_train, y_test = train_test_split(db.data, db.target)

    # Create and train models.
    rf = RandomForestRegressor(n_estimators=100, max_depth=6, max_features=3)
    mlflow.log_params({'model_params' : rf.get_params()})
    rf.fit(X_train, y_train)

    # Use the model to make predictions on the test dataset.
    predictions = rf.predict(X_test)
    mse = mean_squared_error(y_test, predictions)
    mlflow.log_metric('mse', mse)

    signature = infer_signature(X_test, predictions)
    mlflow.sklearn.log_model(rf, "model", signature=signature)

    print("Run ID: {}".format(run.info.run_id))


Run ID: 09e382cef62a4291a36c82e75401dc11


## Model Registry
When you are training and deploying a bunch of models in production, it becomes difficult to track what models are live, what models are challenging and which models are archived. 
MLflow has a really handy tool called the `Model Registry` which keeps track of all of this information. this really helps when it comes to deploying the model and dealing with the ML engineers to get a model into production.

```{note}
The model registry doesn't actually deploy any models into production. It is only a place that lists which models are production ready
```

registered models have a unique name, contain versions, associated transitional stages, model lineage, and other metadata.

## Types of Models in the Registry: 
MLflows model registry has 3 types of models included:

| Name              | Used for                                                          |
| :---------------- | :----------------------------------------------------------------:|
| Staging           |  Models that have been training and are going through testing against the current production model |
| Production        |  Models that have completed testing / review and have been deployed to applications                |
| Archived          |  Models that were in production / staging but are no longer used by any application                |

### How to Move Models to Staging:
Models can be moved to the model registry in 2 ways:
1. In the UI
2. Through the API

In [5]:
from mlflow.tracking import MlflowClient
client = MlflowClient(tracking_uri="sqlite:///mlflow.db")

In [6]:
client.search_experiments()

[<Experiment: artifact_location='/home/ubuntu/sh-mlops-zoomcamp/mlruns/4', creation_time=1685826086193, experiment_id='4', last_update_time=1685826086193, lifecycle_stage='active', name='optuna_test', tags={}>,
 <Experiment: artifact_location='/home/ubuntu/sh-mlops-zoomcamp/mlruns/3', creation_time=1685394893089, experiment_id='3', last_update_time=1685394893089, lifecycle_stage='active', name='exp_from_api_test', tags={}>,
 <Experiment: artifact_location='/home/ubuntu/sh-mlops-zoomcamp/mlruns/1', creation_time=1685052038922, experiment_id='1', last_update_time=1685052038922, lifecycle_stage='active', name='jb-experiment', tags={}>,
 <Experiment: artifact_location='/home/ubuntu/sh-mlops-zoomcamp/mlruns/0', creation_time=1685040306314, experiment_id='0', last_update_time=1685040306314, lifecycle_stage='active', name='Default', tags={}>]

In [7]:
try:
    client.create_experiment(name = 'exp_from_api_test')
except:
    pass

In [8]:
runs = client.search_runs(experiment_ids='1')

In [9]:
#access the run-id
print(f"Run Id: {runs[0].info.run_id}, MSE: {runs[0].data.metrics['mse']}")

Run Id: 09e382cef62a4291a36c82e75401dc11, MSE: 3000.5021268418263


In [10]:
run_id = runs[0].info.run_id
result = mlflow.register_model(
    f"runs:/{run_id}/model", "reg_model_test"
)

Registered model 'reg_model_test' already exists. Creating a new version of this model...
2023/06/05 23:33:57 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: reg_model_test, version 9
Created version '9' of model 'reg_model_test'.


In [11]:
'''
How to transition a model to different stages. The different stages are:
1. Staging
2. Production
3. Archived
'''
client.transition_model_version_stage(
    name="reg_model_test", version=1, stage="Archived"
)

<ModelVersion: aliases=[], creation_timestamp=1685927318810, current_stage='Archived', description=None, last_updated_timestamp=1686008038031, name='reg_model_test', run_id=None, run_link=None, source='093f15e5885449bcb67cce76bb2be6dc', status='READY', status_message=None, tags={}, user_id=None, version=1>

In [12]:
client.transition_model_version_stage(
    name="reg_model_test", version=1, stage="Production"
)

<ModelVersion: aliases=[], creation_timestamp=1685927318810, current_stage='Production', description=None, last_updated_timestamp=1686008038107, name='reg_model_test', run_id=None, run_link=None, source='093f15e5885449bcb67cce76bb2be6dc', status='READY', status_message=None, tags={}, user_id=None, version=1>

model_name = "reg_model_test"
model_version = 7

model = mlflow.pyfunc.load_model(model_uri=f"models:/{model_name}/{model_version}")

model.predict(X_test)[0]

### Additional Model Registry Commands Through The API:

| Name      | Use                                       | Function Name                             |
|-----------|-------------------------------------------|-------------------------------------------|
|Adding / Updating Descriptions               |Adding or Updating a Model Description|MlflowClient().rename_registered_model|
|Renaming Models |Renaming a Model|MlflowClient().update_model_version|
|Serving a Model      |Serving a Model as a service on our Host|export MLFLOW_TRACKING_URI=http://localhost:5000 <br /> mlflow models serve -m "models:/sk-learn-random-forest-reg-model/Production"|

## Deleting Experiments
When you delete an experiment in the MLflow UI it gets sent to the .trash folder but it is not fully deleted. In this state you cannot make a new experiment with the same name as the experiment you just deleted which can be annoying. 

The MLflow CLI has a command that can permenently delete these front-end experiment deletions. Running the following in the command line will delete these experiments:
`mlflow gc --backend-store-uri <PATH>`

Example:
` mlflow gc --backend-store-uri sqlite:///mlflow.db`